# 概要

このノートは、Qiitaの記事

* 15分でできる日本語Word2Vec(https://qiita.com/makaishi2/items/63b7986f6da93dc55edd)

を試したログです。

# 本題

## 青空文庫より「三四郎」をダウンロードし整形

In [ ]:
!curl -LO http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
!unzip 794_ruby_4237.zip
!ls -l sanshiro.txt

In [33]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('sanshiro.txt', "r", "sjis")
text = f.read()
f.close()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])



一

　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌を


取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。






In [34]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

三四郎
京都
ちょっと
用
ある
降りる
ついで
一
する
目
さめる
女
隣
じいさん
話
始める
いる


## 準備したデータを用いてWord2Vecのモデル作成、学習実施

In [35]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [26]:
# 学習ずみモデル保存
model.save("word2vec.model")

## 学習モデルを使い類似単語の調査

In [28]:
# 学習済みモデル読み込み
saved_model = word2vec.Word2Vec.load("word2vec.model")

In [29]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(saved_model.__dict__['wv']['世間'])

[-6.41022325e-01 -8.96081448e-01  1.17433119e+00 -7.06966996e-01
 -2.46490741e+00 -1.31385729e-01  1.03345025e+00 -1.13619041e+00
  1.68432534e+00  2.73926854e+00 -1.47613585e+00  1.85458744e+00
  2.65864044e-01 -6.10771120e-01  2.01002359e-01  2.05743361e+00
  1.47974777e+00  1.96556032e+00  2.99169707e+00 -1.17292255e-01
 -1.38066101e+00 -3.75678957e-01  8.21240962e-01  1.12217879e+00
 -1.22123289e+00 -2.29579255e-01 -3.77498537e-01  2.58674502e+00
 -6.39067769e-01 -1.90476686e-01 -3.45091641e-01  3.04870367e+00
 -5.47300100e-01  1.36662871e-01 -3.14100742e+00  2.94421554e-01
 -2.56990838e+00 -8.83876503e-01 -1.19387627e+00 -2.94832647e-01
 -2.34409511e-01 -9.49867547e-01 -2.87450290e+00 -8.78191829e-01
 -1.49023294e+00  8.55758905e-01 -5.13252735e-01  1.71482980e+00
  1.06291187e+00 -9.08660761e-04  7.88932145e-01 -1.47986925e+00
  9.09207940e-01  2.36083180e-01 -8.90600204e-01 -3.02920914e+00
 -1.72716510e+00  1.00467406e-01 -6.27237856e-01 -1.09999585e+00
 -1.47139680e+00  1.30692

In [31]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = saved_model.wv.most_similar(positive=['京都']) 
for item in ret:
    print(item[0], item[1])

画工 0.35532063245773315
玩具 0.3366878926753998
迷子 0.3066449761390686
便利 0.30430662631988525
とき 0.2998444437980652
テーブル 0.29242438077926636
責任 0.28952527046203613
独身 0.28797459602355957
入口 0.2812340259552002
別れる 0.27822446823120117


/Users/junnakano/anaconda/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
